# get a od matrix and generate activity space parameters

In [ ]:
%pylab inline

import numpy as np
import pandas as pd
from pandas import DataFrame
import math
import statistics
from pathlib import Path

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))



To make it easier for all to load the data with the same path, place it in a sub directory (not in git)

In [ ]:
%ls data

In [ ]:
input_base = Path('data')

odm = pd.read_parquet(input_base / 'od_political_area__stay60.parquet')
display(odm.head(n=2))

In [ ]:
odm = odm.sort_values(['time_bin', 'origin_locationid'])

In [ ]:
def findAllDestsByDateOrg (time_bin, origin, odm):
    dests_org = odm[(odm.time_bin == time_bin) & (odm.origin_locationid == origin)]
    return dests_org

findAllDestsByDateOrg('2020-02-15', 'Bregenz', odm).head()

@Farid: by having multiple cells it might be easier for you to debug the code. I am not 100% sure it is doing what you want, but I hope I could outline how this makes handling the data a lot simpler & faster.

In [ ]:

odm_list = odm.values.tolist()

date_org_pair = sorted(list(set(zip (odm.time_bin, odm.origin_locationid))))
date_org_alldests = [findAllDestsByDateOrg (x, odm_list) for x in date_org_pair]


In [ ]:
# 







def calcActSpcPars (rec):
  xO = float(rec[2])
  yO = float(rec[3])
  dests_info = rec[4]
  
  dests_x = [float(x[0]) for x in dests_info]
  dests_y = [float(x[1]) for x in dests_info]
  dests_c = [float(x[2]) for x in dests_info]
    
  dx2 = [(x-xO)**2 for x in dests_x]
  dy2 = [(y-yO)**2 for y in dests_y]
  dists = [math.sqrt(dx22+dy22) for (dx22, dy22) in zip(dx2, dy2)]

  travel_sum = sum(dests_c)
  dist_sum = sum([c*dist for (c, dist) in zip(dests_c, dists)]) / 1000
  dist_mean = dist_sum / travel_sum
 
  x_mean = sum([c*x for (c, x) in zip(dests_c, dests_x)]) / travel_sum
  y_mean = sum([c*y for (c, y) in zip(dests_c, dests_y)]) / travel_sum

  diff_x = [x-x_mean for x in dests_x]
  diff_y = [y-y_mean for y in dests_y]

  diff_x2 = [x*x for x in diff_x]
  sig_x2 = sum([c*x2 for (c, x2) in zip(dests_c, diff_x2)]) / travel_sum

  diff_y2 = [y*y for y in diff_y]
  sig_y2 = sum([c*y2 for (c, y2) in zip(dests_c, diff_y2)]) / travel_sum

  diff_xy = ([x*y for x, y in zip(diff_x, diff_y)])
  sig_xy = sum([c*xy for (c, xy) in zip(dests_c, diff_xy)]) / travel_sum

  a = math.sqrt(((sig_x2+sig_y2)+math.sqrt((sig_x2-sig_y2)**2 + 4* (sig_xy**2)/2)))/1000
  b = math.sqrt(((sig_x2+sig_y2)-math.sqrt((sig_x2-sig_y2)**2 + 4* (sig_xy**2)/2)))/1000
  teta = (-0.5 * math.atan((2*sig_xy)/(sig_x2-sig_y2))) / math.pi * 180
  area = math.pi * a * b
  shapeIndxex = 1 - math.sqrt(1-((b/a)**2))  
  actSpc_pars = rec[0:4]+[travel_sum, dist_sum, dist_mean]+[a, b, teta, area, shapeIndxex]

  return actSpc_pars





In [ ]:
actSpc_pars = [calcActSpcPars(x) for x in date_org_alldests]
df = DataFrame (actSpc_pars, columns =['Date', 'Origin', 'X Origin', 'Y Origin', 'Num Travels', 'Sum Travels', 'Mean Travels', 'a', 'b', 'teta', 'Area', 'Shape Indxex'])
df.to_csv('d:\\ActivitySpacePars.csv', index=False)
